Importing the useful libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv

## Loading the data

In [70]:
def load_test_data(data_path):
    """Loads data and returns ids (event ids) and X (feature vector)"""
    
    f = open(data_path)
    x = f.readlines()
    
    n = np.shape(x)[0]
    ids = []
    for k in range(n):
        ids.append(k)
        while (x[k][0] != ','):
            x[k] = x[k][1:]
        x[k] = x[k][1:]
    x = pd.DataFrame(x,ids,columns=['Tweets'])
    
    return x

data_path = 'data/twitter-datasets/twitter-datasets/test_data.txt'
x = load_test_data(data_path)

x

,Tweets
0,sea doo pro sea scooter ( sports with the port...
1,<user> shucks well i work all week so now i ca...
2,i cant stay away from bug thats my baby\n
3,<user> no ma'am ! ! ! lol im perfectly fine an...
4,"whenever i fall asleep watching the tv , i alw..."
5,<user> he needs to get rid of that thing ! it ...
6,its whatever . in a terrible mood ( (\n
7,"yesss ! rt <user> <user> thanks jordan , i lov..."
8,my friend <user> text me to check up on me las...
9,<user> #followback please . when will ur #unit...


## Data Analysis

## Word enbeddings matrix

### Generation

### Visualization

## Building the text classifier

### Training texts features

### Linear classifiers

### Predictions

## Output creation and visualization

### Creating the submission file

### Visualization of the results